<a href="https://colab.research.google.com/github/0x1beef/uap/blob/main/src/gofast_ranging.ipynb">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>
<a href="https://kaggle.com/kernels/welcome?src=https://github.com/0x1beef/uap/blob/main/src/gofast_ranging.ipynb">
    <img src="https://kaggle.com/static/images/open-in-kaggle.svg" alt="Open In Kaggle"   />
</a>

# Download Sitrec, set it up for running scripts

In [ ]:
# download only what's needed for the script
!git clone --no-checkout --filter=tree:0 https://github.com/MickWest/sitrec.git
!cd sitrec && git sparse-checkout set --no-cone /src /config* /package* /data/gofast
!cd sitrec && cp config.js.example config.js

In [ ]:
!curl -fsSL https://bun.sh/install | bash

import os
os.environ['PATH'] = "~/.bun/bin:" + os.environ['PATH']

In [ ]:
!cd sitrec && bun install --production

# Get the GoFast data

In [ ]:
%%writefile gofast.js
// note: the Sitrec internal API may change over time
import { initSitch } from "./sitrec/src/indexCommon.js"
import { NodeMan } from "./sitrec/src/Globals.js"
import { Sit } from "./sitrec/src/Globals.js"
import { jetRollFromFrame } from "./sitrec/src/JetStuff.js"
import csv from "./sitrec/src/js/jquery.csv.js"
import fs from "node:fs"

await initSitch("gofast", "SitGoFast.js")

const jetTrack_node = NodeMan.get("jetTrack")
const targetTrack_node = NodeMan.get("LOSTraverseSelect")

var data = [["jet", "target", "jet_roll"]]
for(let frame = 0; frame < Sit.frames; frame++) {
    const pj = jetTrack_node.getValueFrame(frame).position.toArray()
    const pt = targetTrack_node.getValueFrame(frame).position.toArray()
    const jet_roll = jetRollFromFrame(frame)
    data.push([pj, pt, jet_roll])
}

const csv_text = csv.fromArrays(data)
fs.writeFileSync("sitrec_data.csv", csv_text)

In [ ]:
!bun gofast.js

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv("sitrec_data.csv")
df = df.map(lambda p : p if type(p) != str else np.array(p.split(','), np.float64))
df

# Constant Velocity Passive Ranging 

In [ ]:
import numpy as np

# if at time t[i] the camera is at 3D position C[i]
# and the LOS towards the target is the unit vector L[i]
# return the speed and distance to the target at t[0]
# assuming the target's velocity is constant
def constant_velocity_ranging(t, C, L):
    n = len(t)
    # If P[i] is the target's postion at t[i]
    # and d is the distance to the target at t[0] then
    # P[0] = C[0] + d L[0] (1)
    # Given the constant velocity vector V we also have
    # P[i] = P[0] + V (t[i]-t[0]) (2)
    # Since P[i] must be along the line of sight L[i] from C[i]
    # (P[i] - C[i]) x L[i] = 0 (3)
    # Substituting (1),(2) into (3) and rearranging we get
    # d L[0]xL[i] + V x L[i] (t[i]-t[0]) = (C[i]-C[0]) x L[i]
    # This is a system of 3*(n-1) linear equations which we
    # represent as A X = B where X = (d,Vx,Vy,Vz)
    # and solve for X using linear least squares.
    (eqs, vars) = (3*(n-1), 4)
    A = np.zeros(shape=[eqs,vars])
    B = np.zeros(shape=[eqs,1])
    for i in range(1, n):
        eq = 3*(i-1)
        L0xLi = np.cross(L[0], L[i])
        dt = t[i]-t[0]
        # V x L = (Vy Lz - Vz Ly, Vz Lx - Vx Lz, Vx Ly - Vy Lz)
        A[eq:eq+3] = np.array([
            [ L0xLi[0],  0,          dt*L[i,2], -dt*L[i,1] ],
            [ L0xLi[1], -dt*L[i,2],  0,          dt*L[i,0] ],
            [ L0xLi[2],  dt*L[i,1], -dt*L[i,0],  0 ]
        ])
        B[eq:eq+3,0] = np.cross(C[i]-C[0], L[i])
    ret = np.linalg.lstsq(A,B,rcond=None)
    X = ret[0]
    d = X[0,0]
    V = X[1:]
    v = np.linalg.norm(V)
    debug_info = { 'A': A, 'B': B, 'ret': ret }
    return (v, d, debug_info)

# Tracking Error Model

In [ ]:
import math

def get_heading_from_az_el(az,el):
    (az,el) = (math.radians(az), math.radians(el))
    x = math.cos(el) * math.sin(az)
    y = math.sin(el)
    z = math.cos(el) * math.cos (az)
    return np.array([x,y,-z])

def get_az_el_from_heading(v):
    (vx,vy,vz) = (v[0],v[1],v[2])
    el = math.atan2(vy, math.sqrt(vz*vz + vx*vx))
    az = math.atan2(vx,-vz)
    return (math.degrees(az), math.degrees(el))

# return a randomly adjusted line of sight (L), given
# the standard deviation of the tracking error in pixels
def add_tracking_error(L, pixels):
    FOV = 0.7
    img_size = 480 # original sensor size
    # angle corresponding to the given number of pixels:
    err = FOV / img_size * pixels
    # we sample two numbers from a normal distribution
    # whose standard deviation is 'err', centered around 0
    E = np.random.normal(0.0, err, 2)
    (az,el) = get_az_el_from_heading(L) + E
    return get_heading_from_az_el(az,el)

# calculate the noisy lines of sight only once
# and store the results in the dataframe
def add_tracking_errors(track_error):
    def los_err_for_row(r):
        return add_tracking_error(r.target - r.jet, track_error)
    df['Lerr'] = df.apply(los_err_for_row, axis=1)

def frame_to_time(frame):
    return frame / 29.97
def time_to_frame(time):
    return time * 29.97

# for the given set of frame indices
# return the frame times, camera positions and lines of sight.
def get_tCL(frames):
    n = len(frames)
    t = np.zeros(shape=[n])
    C = np.zeros(shape=[n,3])
    L = np.zeros(shape=[n,3])
    for i in range(n):
        t[i] = frame_to_time(frames[i])
        data = df.loc[frames[i]]
        C[i] = data.jet
        L[i] = data.Lerr # from add_tracking_errors 
    return (t,C,L)

def print_mat(name, M):
    print(name, M.shape, '=\n', M)
    
def debug_tCL(t,C,L):
    print_mat('t',t)
    print_mat('C',C)
    print_mat('L',L)

def debug_lstsq(info):
    (A,B,ret) = (info['A'],info['B'],info['ret'])
    X = ret[0]
    print_mat('A',A)
    print_mat('B',B)
    print_mat('X',X)
    print(ret)
    print('cond', np.linalg.cond(A))
    print(np.linalg.norm(B - A @ X))

# convert meters to nautical miles
def m_to_nm(meters):
    return meters / 1852

# convert meters per second to knots
def mps_to_knots(meters_per_sec):
    return meters_per_sec * 1.94384

# get an average range estimate for a set of frames
def get_range_for_frames(frames, debug = 0):
    (t,C,L) = get_tCL(frames)
    if debug >= 3: debug_tCL(t,C,L)
    (v,d,debug_info) = constant_velocity_ranging(t,C,L)
    (v,d) = (mps_to_knots(v), m_to_nm(d))
    if debug >= 2: debug_lstsq(debug_info)
    if debug >= 1: print(f'd = {d:.2f} nm, v = {v:.2f} knots')
    return d

# Plot the Passive Ranging for GoFast

In [ ]:
import matplotlib.pyplot as plt

def plot_range():
    first_track = 338 # first frame where the target starts being tracked
    first_range = 370 # first frame where the range is displayed
    # plot the estimates between these frames
    (start_frame, end_frame) = (first_range, 1030)
    # use 'count' number of frames for the estimate, separated by 'step' frames
    # the estimate for time t will depend on data from step*count frames before t
    (step, count) = (1,60)
    # the standard deviation of the tracking error in pixels
    track_error = 0.25

    add_tracking_errors(track_error)
    
    # get a range estimate starting from frame 'f'
    def get_range(f, debug = 0):
        frames = [f - i*step for i in range(count)]
        frames = [i for i in frames if first_track <= i <= end_frame]
        return get_range_for_frames(frames, debug)

    #get_range(start_frame, debug = 3)
    #for i in range(start_frame,end_frame,50):
    #    get_range(i, debug = 1)
    
    fig, ax1 = plt.subplots()
    title = f'passive ranging (σ = {track_error}, n = {count}'
    if step != 1:
        title += f', step = {step}'
    title += ')'
    ax1.set_title(title)
    ax1.set_xlabel('frames', loc='right')
    ax1.set_ylabel('nm')
    ax1.grid()

    df_plot_all = df.loc[max(start_frame - step * count, first_track):end_frame]
    df_plot_est = df.loc[start_frame:end_frame]
    
    def sitrec_range(r):
        return m_to_nm(np.linalg.norm(r.target - r.jet))
    ax1.plot(df_plot_all.apply(sitrec_range, axis=1), label = 'real range')
    ax1.plot(df_plot_est.apply(lambda r: get_range(r.name), axis=1),
        label = 'passive range')
    
    ax2 = ax1.twinx()
    ax2.set_ylabel('degrees')
    ax2.plot(df_plot_all.jet_roll, label='bank angle', color='red')
    
    lines = ax1.get_lines() + ax2.get_lines()
    labels = [l.get_label() for l in lines]
    ax2.legend(lines, labels, loc=0)
    
    ax3 = ax1.secondary_xaxis(location='top',
        functions=(frame_to_time, time_to_frame))
    ax3.set_xlabel('time [s]', loc='right')
    
    plt.show()

plot_range()